In [3]:
import cv2, os
import numpy as np

In [ ]:
INPUT_PATH = "<path>/data/cholecseg8k/"
OUT_PATH = "<path>/cholecseg8k_preprocessed_2/"

In [5]:
color_class_mapping={
                            #(127, 127, 127): 0,
                            (140, 140, 210): 1,     #Abdominal wall        
                            (114, 114, 255): 2,     #Liver                 
                            #(156, 70, 231): 3,     #Gastrointestinal tract
                            (75, 183, 186): 4,      #Fat                   
                            (0, 255, 170): 5,       #Grasper               
                            #(0, 85, 255): 6,       #Connective tissue     
                            #(0, 0, 255): 7,        #Blood                 
                            #(0, 255, 255): 8,      #Cystic duct           
                            #(184, 255, 169): 9,    #L-hook electrocautery   
                            (165, 160, 255): 10,    #Gallbladder             
                            #(128, 50, 0): 11,      #Heptatic vein         
                            #(0, 74, 111): 12       #Liver ligament        
                            }

color_classes = [k for k in color_class_mapping.keys()]

In [7]:
for patient in os.listdir(INPUT_PATH):
    print(patient)
    if not os.path.isdir(os.path.join(INPUT_PATH, patient)):
        continue
    for f in os.listdir(os.path.join(INPUT_PATH, patient)):
        first_frame = int(f[len("videoXX_"):])
        path = os.path.join(INPUT_PATH, patient, f)

        imgs = []
        lbls = []
        for frame in range(first_frame, first_frame + 80):
            label = cv2.imread(os.path.join(path, f"frame_{frame}_endo_color_mask.png"))
            lbls.append(label)
            image = cv2.imread(os.path.join(path, f"frame_{frame}_endo.png"))
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            imgs.append(image)
        
        imgs = np.array(imgs)
        lbls = np.array(lbls)
        
        mean = [0.485, 0.456, 0.406]
        std = [0.229, 0.224, 0.225]

        imgs = imgs.astype(np.float32)
        imgs /= 255.0

        imgs -= mean
        imgs /= std

        imgs = imgs.transpose(3, 1, 2, 0)#DHWC -> CHWD
        lbls = lbls.transpose(1, 2, 0, 3)#DHWC -> HWDC

        new_labels = np.zeros((lbls.shape[0], lbls.shape[1], lbls.shape[2], len(color_classes)), dtype=np.uint8)


        for i, k in enumerate(color_classes):
            mask = np.all(lbls == k, axis=-1)
            new_labels[mask, i] = 1

        os.makedirs(os.path.join(OUT_PATH, patient, f), exist_ok=True)
        np.save(os.path.join(OUT_PATH, patient, f, "video.npy"), imgs)
        np.save(os.path.join(OUT_PATH, patient, f, "segmentation.npy"), new_labels)


video43
video18
video28
video09
readme
video20
video48
video55
video37
video26
video35
video25
video24
video01
video52
video12
video17
video27
